# False Positive Reduction (FPR) - CNN Classifier

In this section, we use a simple Convolutional Neural Network (CNN) to perform False Positive Reduction (FPR). The workflow includes loading the data, preprocessing the Regions of Interest (ROIs), training the model, and evaluating performance.

---

## 1. Load the Data

First, we load the tumor candidate images and their corresponding labels from the `.pkl` files.


In [31]:
import pickle

with open('./Brainhack/final_project/Image_Tumor.pkl', 'rb') as f:
   image = pickle.load(f)

with open('./Brainhack/final_project/label_Tumor.pkl', 'rb') as f:
   label1 = pickle.load(f)

## 2. Preprocess the ROI Images
We resize all ROI images to a fixed size (60x60) using bicubic interpolation. If the image is RGB, we convert it to grayscale.

## 3. Shuffle and Split the Dataset
Shuffle the dataset and split it into training and validation sets (90% train, 10% validation).

## 4. Define the CNN Model
We define a CNN architecture with two convolutional blocks and three fully connected layers.

## 5. Train the Model
We train the CNN using cross-entropy loss and the Adam optimizer.

## 6. Save the Model
Finally, we save the trained model for later use.

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pickle
import numpy as np
import torch.nn.functional as F

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Initialize arrays to store image data and labels
N = len(label1['output'])
image_data = np.zeros((N, 60, 60))
label = np.zeros((N, 1))

for i in range(N):
    # Save label
    label[i, :] = label1['output'][i]

    # Convert image to tensor and add batch & channel dimensions
    img = image['output'][i]
    if img.ndim == 2:  # Grayscale
        img_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # shape: (1, 1, H, W)
    elif img.ndim == 3:  # RGB
        img_tensor = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)  # (1, C, H, W)
    else:
        raise ValueError(f"Unexpected image shape: {img.shape}")

    # Resize to 60x60 using bicubic interpolation
    img_resized = F.interpolate(img_tensor, size=(60, 60), mode='bicubic', align_corners=False)

    # Remove batch & channel dims and store as numpy
    if img_resized.shape[1] == 1:
        image_data[i, :, :] = img_resized.squeeze().numpy() / 255.0
    else:
        # If RGB, convert to grayscale or keep first channel
        image_data[i, :, :] = img_resized[0, 0, :, :].numpy() / 255.0


print(label.shape)


#print(label[:10])

# Shuffle and split the data into training and validation sets
N1 = len(label)
np.random.seed(20)
ind = np.random.permutation(N1)
#print(N1)

#print(ind)
image_data = image_data[ind, :, :]
label = label[ind, :]

print(label[:10])

#print(image_data)

# Normalize image data and convert labels to integer type
image_data = image_data.reshape(-1, 1, 60, 60).astype('float32')
label = np.squeeze(label).astype('int64')

print(label[1000:1010])
# Define the split ratios for training and validation data
training_split = 0.9
num_images = image_data.shape[0]
train_size = int(training_split * num_images)
val_size = num_images - train_size

# Create PyTorch tensors for training and validation data
image_data1_train = torch.tensor(image_data[:train_size])
image_data1_val = torch.tensor(image_data[val_size:])
label1_train = torch.tensor(label[:train_size])
label1_val = torch.tensor(label[val_size:])

# Create datasets and data loaders for training and validation sets
train_dataset = data.TensorDataset(image_data1_train, label1_train)
val_dataset = data.TensorDataset(image_data1_val, label1_val)

train_loader = data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = data.DataLoader(val_dataset, batch_size=16, shuffle=False)

print(label1_train[:10])

# Define the CNN model architecture
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Define the layers of the CNN
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 15 * 15, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):
        # Define the forward pass
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # Output layer
        return x

# Initialize the model and move it to the appropriate device
model = CNNModel().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the model parameters
        
        running_loss += loss.item()
    
    # Print loss after each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluate the model
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print validation accuracy
print(f'\nValidation Accuracy: {100 * correct / total:.2f}%')


# Save the trained model
torch.save(model.state_dict(), './Brainhack/final_project/FPR_Tumor.pth')



Using device: cpu
(1397, 1)
[[1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]]
[1 0 1 1 1 1 1 1 1 0]
tensor([1, 0, 1, 1, 1, 1, 0, 1, 0, 1])
Epoch [1/5], Loss: 0.2668
Epoch [2/5], Loss: 0.1948
Epoch [3/5], Loss: 0.1863
Epoch [4/5], Loss: 0.1216
Epoch [5/5], Loss: 0.1120

Validation Accuracy: 95.15%
